# Lab

Using the following code as a basis, use async methods to fetch the temperatures in the given cities:

In [1]:
APPID = '10d4440bbaa8581bb8da9bd1fbea5617'
# UNITS = 'metric'
UNITS = 'imperial'
cities = [
    'Boulder', 'Atlanta', 'San Francisco',
    'Reno', 'Honolulu', 'Zurich', 'Dubai',
    'Dublin','Stuttgart', 'Rome', 'Singapore', 
    'Bangalore', 'Hyderabad', 'Seattle',
]

from aiohttp_requests import requests

async def get_temp(city, units=UNITS, appid=APPID):
    url = 'http://api.openweathermap.org/data/2.5/weather'
    resp = await requests.get(url, params={
        'q': city,
        'units': units,
        'appid': appid,        
    })
    resp.raise_for_status()
    json_data = await resp.json()
    temp = json_data['main']['temp']
    return temp

In [13]:
import asyncio

coroutines = [get_temp(city) for city in cities]

coroutines

[<coroutine object get_temp at 0x7f1092b0fe40>,
 <coroutine object get_temp at 0x7f1092b0fdc0>,
 <coroutine object get_temp at 0x7f1092b0f340>,
 <coroutine object get_temp at 0x7f1092b0fec0>,
 <coroutine object get_temp at 0x7f1092b0ff40>,
 <coroutine object get_temp at 0x7f1092b19040>,
 <coroutine object get_temp at 0x7f1092b190c0>,
 <coroutine object get_temp at 0x7f1092b19140>,
 <coroutine object get_temp at 0x7f1092b191c0>,
 <coroutine object get_temp at 0x7f1092b192c0>,
 <coroutine object get_temp at 0x7f1092b19340>,
 <coroutine object get_temp at 0x7f1092b193c0>,
 <coroutine object get_temp at 0x7f1092b19440>,
 <coroutine object get_temp at 0x7f1092b194c0>]

In [14]:
result = await asyncio.gather(*coroutines)
# result = await asyncio.gather(coroutines[0], coroutines[1], ...)

In [4]:
result

[36.55,
 68.49,
 60.55,
 57.18,
 80.73,
 45.73,
 82.15,
 60.49,
 46.09,
 68.54,
 77,
 67.82,
 72.91,
 53.78]

In [5]:
cities

['Boulder',
 'Atlanta',
 'San Francisco',
 'Reno',
 'Honolulu',
 'Zurich',
 'Dubai',
 'Dublin',
 'Stuttgart',
 'Rome',
 'Singapore',
 'Bangalore',
 'Hyderabad',
 'Seattle']

In [6]:
dict(zip(cities, result))

{'Boulder': 36.55,
 'Atlanta': 68.49,
 'San Francisco': 60.55,
 'Reno': 57.18,
 'Honolulu': 80.73,
 'Zurich': 45.73,
 'Dubai': 82.15,
 'Dublin': 60.49,
 'Stuttgart': 46.09,
 'Rome': 68.54,
 'Singapore': 77,
 'Bangalore': 67.82,
 'Hyderabad': 72.91,
 'Seattle': 53.78}

In [7]:
async def serialized():
    result = []
    for city in cities:
        result.append(await get_temp(city))
    return result

In [11]:
result = await serialized()

In [9]:
result

[36.55,
 68.56,
 60.57,
 57.18,
 80.29,
 45.61,
 82.15,
 60.19,
 46.71,
 68.45,
 77.04,
 67.82,
 72.91,
 53.51]

How about queues?

In [15]:
from asyncio import Queue

In [23]:
q = Queue()

async def worker(i, q):
    while True:
        city = await q.get()
        temp = await get_temp(city)
        print(i, city, temp)

In [24]:
for city in cities:
    await q.put(city)

In [25]:
workers = [worker(i, q) for i in range(4)]

In [26]:
await asyncio.gather(*workers)

0 Boulder 37.76
2 San Francisco 60.55
1 Atlanta 68.52
3 Reno 57.15
0 Honolulu 80.38
2 Zurich 45.73
1 Dubai 82.15
3 Dublin 60.58
2 Rome 68.47
1 Singapore 77
1 Seattle 53.78
2 Hyderabad 72.91
0 Stuttgart 46.69
3 Bangalore 67.82


CancelledError: 